In [5]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import pickle
import functions.calibration as cal


### Import and Process Data

In [ ]:
df0  = pd.read_csv("data generation/test csv/saved_test_dataframe0.csv")
df1  = pd.read_csv("data generation/test csv/saved_test_dataframe1.csv")
df2  = pd.read_csv("data generation/test csv/saved_test_dataframe2.csv")
df3  = pd.read_csv("data generation/test csv/saved_test_dataframe3.csv")
df4  = pd.read_csv("data generation/test csv/saved_test_dataframe4.csv")

df = pd.concat([df0, df1, df2, df3,  df4], ignore_index=True)

df.rename(columns={
    'F0': 'forward_price',
    'IV': 'impl_volatility',
    'K': 'strike_price'
}, inplace=True)

chunks = cal.split_dataframe(df, 41)


In [7]:
# Trim to +-200bps
chunks = [chunk.iloc[4:-4].reset_index(drop=True) for chunk in chunks]

cleaned_chunks = []
for chunk in chunks:
    chunk['impl_volatility'] = chunk['impl_volatility'].replace([np.inf, -np.inf], np.nan)

    # Keep only rows where impl_volatility is a finite, positive number
    chunk = chunk[
        chunk['impl_volatility'].notna() &
        (chunk['impl_volatility'] > 1e-8)
    ]
    cleaned_chunks.append(chunk)

cleaned_chunks = [chunk.reset_index(drop=True) for chunk in cleaned_chunks]

# Keep only chunks with 10 or more rows
cleaned_chunks = [chunk for chunk in cleaned_chunks if len(chunk) >= 10]

In [ ]:
# Save the cleaned dataframes
with open("cleaned_test_chunks.pkl", "wb") as f:
    pickle.dump(cleaned_chunks, f)